In [1]:
%%script C:\Users\Jan Catherine\AppData\Local\Programs\Python\Python38\python.exe
import sys
print(sys.version)

3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [2]:
pwd

'C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN'

In [3]:
import os
os.chdir('../../Data_Collection_02/02_Preprocessing/05_CNN')

In [4]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy


from sklearn.model_selection import KFold

#import pydot

In [5]:
dataset = pd.read_csv('02_All_Train.csv', encoding='latin-1')
testdata = pd.read_csv('02_All_Test.csv', encoding='latin-1')

train_data = dataset
test_data = testdata

test_data.dropna(axis = 0, how ='any',inplace=True) 

In [6]:
new_model1 = tf.keras.models.load_model('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Title')
new_model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4_1 (Embedding)   (None, 90, 64)            750016    
                                                                 
 conv1d_4_1 (Conv1D)         (None, 88, 128)           24704     
                                                                 
 global_max_pooling1d_4_1 (  (None, 128)               0         
 GlobalMaxPooling1D)                                             
                                                                 
 dropout_4_1 (Dropout)       (None, 128)               0         
                                                                 
 dense_4_1 (Dense)           (None, 2)                 258       
                                                                 
Total params: 774978 (2.96 MB)
Trainable params: 774978 (2.96 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [7]:
import json
with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Title\\Title.json') as json_file1:
    json_string1 = json.loads(json_file1.read())
tokenizer1 = tf.keras.preprocessing.text.tokenizer_from_json(json_string1)

In [8]:
new_model2 = tf.keras.models.load_model('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Desc')
new_model2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 50, 64)            1280064   
                                                                 
 conv1d_4 (Conv1D)           (None, 48, 128)           24704     
                                                                 
 global_max_pooling1d_4 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1305026 (4.98 MB)
Trainable params: 1305026 (4.98 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [9]:
import json
with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Desc\\Desc.json') as json_file2:
    json_string2 = json.loads(json_file2.read())
tokenizer2 = tf.keras.preprocessing.text.tokenizer_from_json(json_string2)

In [10]:
x_test1  = np.array( tokenizer1.texts_to_sequences(test_data['Title'].tolist()) )
x_test1 = pad_sequences(x_test1, padding='post', maxlen=90)


C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_14904\3328888694.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test1  = np.array( tokenizer1.texts_to_sequences(test_data['Title'].tolist()) )


In [11]:
x_test2  = np.array( tokenizer2.texts_to_sequences(test_data['Description'].tolist()) )
x_test2 = pad_sequences(x_test2, padding='post', maxlen=50)


C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_14904\4054164706.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test2  = np.array( tokenizer2.texts_to_sequences(test_data['Description'].tolist()) )


In [12]:
from keras. models import Model
from keras.layers import *

In [13]:
merged_model=tf.keras.layers.Average()([new_model1.output, new_model2.output])

final_model = tf.keras.models.Model(inputs=[new_model1.output, new_model2.output], outputs=merged_model)
final_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['CategoricalAccuracy'])

In [14]:
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 2)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 2)]                  0         []                            
                                                                                                  
 average (Average)           (None, 2)                    0         ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
Total params: 0 (0.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 0 (0.00 Byte

In [15]:
print("Generate predictions for all samples")
predictions1 = new_model1.predict(x_test1)
print(predictions1)

predict_results1 = predictions1.argmax(axis=1)

Generate predictions for all samples
51/51 [==============================] - 0s 3ms/step
[[0.9986129  0.00138709]
 [0.99433863 0.00566135]
 [0.99058545 0.00941452]
 ...
 [0.00372173 0.9962783 ]
 [0.9556978  0.04430221]
 [0.9832606  0.01673941]]


In [16]:
print("Generate predictions for all samples")
predictions2 = new_model2.predict(x_test2)
print(predictions2)
predict_results2 = predictions2.argmax(axis=1)

Generate predictions for all samples
51/51 [==============================] - 0s 2ms/step
[[0.98725384 0.01274614]
 [0.99856377 0.00143621]
 [0.9836872  0.01631275]
 ...
 [0.00634583 0.9936542 ]
 [0.9606546  0.03934541]
 [0.9776184  0.02238159]]


In [17]:
print("Generate predictions for all samples")
predictions = final_model.predict([predictions1, predictions2])
print(predictions)
predict_results = predictions.argmax(axis=1)

Generate predictions for all samples
51/51 [==============================] - 0s 988us/step
[[0.9929334  0.00706661]
 [0.9964512  0.00354878]
 [0.98713636 0.01286364]
 ...
 [0.00503378 0.99496627]
 [0.9581762  0.04182381]
 [0.9804395  0.0195605 ]]


In [18]:
test_data['pred_fake']= predict_results
test_data['pred_fake'] = np.where((test_data.pred_fake == '0'),0,test_data.pred_fake)
test_data['pred_fake'] = np.where((test_data.pred_fake == '1'),1,test_data.pred_fake)

In [19]:
print(type(test_data['Fake']))
test_data['Fake'].head()

<class 'pandas.core.series.Series'>


0    0
1    0
2    0
3    0
4    0
Name: Fake, dtype: int64

In [20]:
print(type(test_data['pred_fake']))
test_data['pred_fake'].head()

<class 'pandas.core.series.Series'>


0    0
1    0
2    0
3    0
4    0
Name: pred_fake, dtype: int64

In [21]:
labels = [0, 1]
    
print(classification_report(test_data['Fake'].tolist(),test_data['pred_fake'].tolist(),labels=labels))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      1428
           1       0.87      0.71      0.78       184

    accuracy                           0.95      1612
   macro avg       0.92      0.85      0.88      1612
weighted avg       0.95      0.95      0.95      1612



In [22]:
final_model.save('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\All_No_URL') 


INFO:tensorflow:Assets written to: C:\Users\Jan Catherine\Documents\CMSC Notes\CMSC 190 Part 2\Codes\CNN\All_No_URL\assets


INFO:tensorflow:Assets written to: C:\Users\Jan Catherine\Documents\CMSC Notes\CMSC 190 Part 2\Codes\CNN\All_No_URL\assets


In [23]:
#import json
#with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\All_No_URL\\All_No_URL.json', 'w') as outfile:
#    json.dump(outfile)